# Setup

In [6]:
import numpy as np
import pandas as pd
import os

In [7]:
data_processed = "../../data/processed/"
data_interim = "../../data/interim/"
data_external = "../../data/external/"
path = data_external+"graphite/"

edgetypes = pd.read_csv(path+"graphite_edgeTypes.tsv",sep="\t")
pathways = pd.read_csv(path+"graphite_pathways.tsv",sep="\t")
reactome_edges = pd.read_csv(path+"graphite_reactome_edges.tsv",sep="\t")
graph_node_data = pd.read_csv(data_processed+"graph_data_nohubs/nohub_graph_node_data.csv")
graph_edge_data = pd.read_csv(data_processed+"graph_data_nohubs/nohub_graph_edge_data.csv")

In [8]:
reactome_edges

,pathwayId,src,dest,type,id
0,R-HSA-1059683,9021,3569,2,9021:3569
1,R-HSA-1059683,9021,3570,2,9021:3570
2,R-HSA-1059683,9021,3716,2,9021:3716
3,R-HSA-1059683,9021,7297,2,9021:7297
4,R-HSA-1059683,3717,9021,2,3717:9021
...,...,...,...,...,...
1913320,R-HSA-997272,9568,94235,2,9568:94235
1913321,R-HSA-997272,3761,94235,2,3761:94235
1913322,R-HSA-997272,3759,94235,2,3759:94235
1913323,R-HSA-997272,3768,94235,2,3768:94235


In [9]:
pathways

,pathwayId,pathwayTitle
0,R-HSA-1059683,Interleukin-6 signaling
1,R-HSA-109581,Apoptosis
2,R-HSA-109582,Hemostasis
3,R-HSA-109606,Intrinsic Pathway for Apoptosis
4,R-HSA-109703,PKB-mediated events
...,...,...
2335,R-HSA-9675135,Diseases of DNA repair
2336,R-HSA-9683612,Maturation of protein M
2337,R-HSA-9694594,Maturation of protein M
2338,R-HSA-975574,Reactions specific to the hybrid N-glycan synt...


In [12]:
new_edges = reactome_edges[["src","dest"]].rename(columns={"src":"x_id","dest":"y_id"})
new_edges["x_type"] = "gene_protein"
new_edges["y_type"] = "gene_protein"
new_edges["edge_type"] = "pathway_edge"
new_edges["edge_source"] = "reactome"

In [13]:
new_edges

,x_id,y_id,x_type,y_type,edge_type,edge_source
0,9021,3569,gene_protein,gene_protein,pathway_edge,reactome
1,9021,3570,gene_protein,gene_protein,pathway_edge,reactome
2,9021,3716,gene_protein,gene_protein,pathway_edge,reactome
3,9021,7297,gene_protein,gene_protein,pathway_edge,reactome
4,3717,9021,gene_protein,gene_protein,pathway_edge,reactome
...,...,...,...,...,...,...
1913320,9568,94235,gene_protein,gene_protein,pathway_edge,reactome
1913321,3761,94235,gene_protein,gene_protein,pathway_edge,reactome
1913322,3759,94235,gene_protein,gene_protein,pathway_edge,reactome
1913323,3768,94235,gene_protein,gene_protein,pathway_edge,reactome


Nodos nuevos

In [28]:
new_nodes = pd.Series(pd.concat([new_edges.x_id,new_edges.y_id]).unique(),name="node_id").astype(str)
old_nodes = graph_node_data[graph_node_data.node_type == "gene_protein"].node_id
not_shared = set(new_nodes.values) - set(old_nodes.values)

In [82]:
not_shared

{'100653049',
 '102723532',
 '102724334',
 '102725035',
 '107080638',
 '107987462',
 '118142757',
 '156377',
 '222236'}

In [63]:
manual_input = {"100653049":"LOC100653049","102723532":"LOC102723532","102724334":"LOC102724334","102725035":"LOC102725035","107080638":"TBC1D7-LOC100130357","107987462":"LOC107987462","118142757":"LOC118142757","222236":"NAPEPLD"}
remove = "156377"

In [94]:
temp_df = pd.DataFrame({"node_id":manual_input.keys(),"node_name":manual_input.values(),"node_type":"gene_protein","node_source":"reactome","degree_gda":0,"degree_pp":0,"degree_dd":0})
temp_df

,node_id,node_name,node_type,node_source,degree_gda,degree_pp,degree_dd
0,100653049,LOC100653049,gene_protein,reactome,0,0,0
1,102723532,LOC102723532,gene_protein,reactome,0,0,0
2,102724334,LOC102724334,gene_protein,reactome,0,0,0
3,102725035,LOC102725035,gene_protein,reactome,0,0,0
4,107080638,TBC1D7-LOC100130357,gene_protein,reactome,0,0,0
5,107987462,LOC107987462,gene_protein,reactome,0,0,0
6,118142757,LOC118142757,gene_protein,reactome,0,0,0
7,222236,NAPEPLD,gene_protein,reactome,0,0,0


In [120]:
new_df = pd.merge(graph_node_data,new_nodes,how="outer").sort_values(by="node_index")
new_df = pd.concat([new_df,temp_df]).dropna(subset="node_name").sort_values(by="node_index").reset_index(drop=True).drop(columns="node_index")
new_df["node_index"] = new_df.index

In [121]:
new_df

,node_id,node_name,node_type,node_source,comunidades_infomap,comunidades_louvain,degree_gda,degree_pp,degree_dd,node_index
0,1,A1BG,gene_protein,disgenet,NaN,NaN,2.0,12.0,0.0,0
1,10,NAT2,gene_protein,disgenet,NaN,NaN,39.0,6.0,0.0,1
2,100,ADA,gene_protein,disgenet,NaN,NaN,18.0,5.0,0.0,2
3,1000,CDH2,gene_protein,disgenet,NaN,NaN,28.0,15.0,0.0,3
4,10000,AKT3,gene_protein,disgenet,NaN,NaN,9.0,39.0,0.0,4
...,...,...,...,...,...,...,...,...,...,...
35842,102725035,LOC102725035,gene_protein,reactome,NaN,NaN,0.0,0.0,0.0,35842
35843,107080638,TBC1D7-LOC100130357,gene_protein,reactome,NaN,NaN,0.0,0.0,0.0,35843
35844,107987462,LOC107987462,gene_protein,reactome,NaN,NaN,0.0,0.0,0.0,35844
35845,118142757,LOC118142757,gene_protein,reactome,NaN,NaN,0.0,0.0,0.0,35845


In [123]:
new_edges

,x_id,y_id,x_type,y_type,edge_type,edge_source
0,9021,3569,gene_protein,gene_protein,pathway_edge,reactome
1,9021,3570,gene_protein,gene_protein,pathway_edge,reactome
2,9021,3716,gene_protein,gene_protein,pathway_edge,reactome
3,9021,7297,gene_protein,gene_protein,pathway_edge,reactome
4,3717,9021,gene_protein,gene_protein,pathway_edge,reactome
...,...,...,...,...,...,...
1913320,9568,94235,gene_protein,gene_protein,pathway_edge,reactome
1913321,3761,94235,gene_protein,gene_protein,pathway_edge,reactome
1913322,3759,94235,gene_protein,gene_protein,pathway_edge,reactome
1913323,3768,94235,gene_protein,gene_protein,pathway_edge,reactome


In [129]:
merged_edges = pd.merge(new_edges.astype({"x_id":str,"y_id":str}),new_df[["node_id","node_index"]],left_on="x_id",right_on="node_id",how="left").drop(columns="node_id").rename(columns={"node_index":"x_index"})
merged_edges = pd.merge(new_edges.astype({"x_id":str,"y_id":str}),merged_edges[["node_id","node_index"]],left_on="y_id",right_on="node_id",how="left").drop(columns="node_id").rename(columns={"node_index":"y_index"})
merged_edges

KeyError: "None of [Index(['node_id', 'node_index'], dtype='object')] are in the [columns]"